In [127]:
import os
import itertools

import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit



In [153]:
def fpath_ktable():
    molecule = 'h2o'
    band = '1'
    dv = .005
    nv = 200
    commitnumber = '9d81085'
    
    atmpro = 'mls'
    tsfc = 294
    
    param = bestfit.kdist_params(molecule='h2o', band='1')
    param['molecule'] = molecule
    param['band'] = band
    param['dv'] = dv
    param['nv'] = nv
    param['commitnumber'] = commitnumber
    param['atmpro'] = atmpro
    param['tsfc'] = tsfc
    
    fpath = os.path.join(pipeline.get_dir_case(param, 
                                               setup=setup_bestfit),
                         'ktable.dat')
    return fpath



def load_ktable(fpath):
    '''
    Returns k-table in the same format as in lblnew.f
    
    Parameter
    ---------
    fpath: string
        Path to the file 'ktable.dat', generated by "lblnew.f".
    ktable: array (number of (p, t) pairs, number of g-intervals)
        Absorption coefficient values calculated at selected 
        (pressure, temperature) pairs.
    '''
    df = pd.read_csv(fpath, sep=r'\s+')
    df = df.set_index('g')
    ng = len(df.index.value_counts())
    ktable = df['k'].values.reshape(ng, -1).T
    return ktable



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    lines = []
    for chunk in chunks:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
    
    return lines
    

    
def ktable_to_F77(ktable):
    npt, ng = ktable.shape
    
    num_values_per_line = 5
    
    last_line = '/)'
    
    lines = []
    for ig in range(ng):
        first_line = 'ktable(:, {}) = (/'.format(ig + 1)
        
        lines_ig = vector_to_F77list(ktable[:, ig], 
                                 num_values_per_line=num_values_per_line)
        
        lines_ig = [first_line] + lines_ig + [last_line]
        
        lines_ig_amp = []
        lines_ig_amp.append(6 * ' ' + lines_ig[0])
        for l in lines_ig[1:]:
            lines_ig_amp.append(5 * ' ' + '&' + l)
            
        lines.extend(lines_ig_amp)
    
        chunks = into_chunks(ktable[:, ig], num_values_per_line)

    fortran = '\n'.join(lines)
    return fortran

In [154]:
ktable = load_ktable(fpath_ktable())

print(ktable_to_F77(ktable))

      ktable(:, 1) = (/
     &   2.587281e-18,   2.672090e-18,   2.743974e-18,   2.802196e-18,   2.848542e-18,
     &   2.990524e-18,   3.100658e-18,   3.195518e-18,   3.274706e-18,   3.338760e-18,
     &   3.451818e-18,   3.597453e-18,   3.725017e-18,   3.834555e-18,   3.924876e-18,
     &   3.990309e-18,   4.181892e-18,   4.352324e-18,   4.501040e-18,   4.627743e-18,
     &   4.624376e-18,   4.873575e-18,   5.098390e-18,   5.297282e-18,   5.469898e-18,
     &   5.354138e-18,   5.676051e-18,   5.970536e-18,   6.234506e-18,   6.466616e-18,
     &   6.196142e-18,   6.608128e-18,   6.990227e-18,   7.337011e-18,   7.645573e-18,
     &   7.165337e-18,   7.687812e-18,   8.179060e-18,   8.630261e-18,   9.036139e-18,
     &   8.266644e-18,   8.924026e-18,   9.550960e-18,   1.013374e-17,   1.066354e-17,
     &   9.573109e-18,   1.039258e-17,   1.118397e-17,   1.192721e-17,   1.260881e-17,
     &   1.152482e-17,   1.253472e-17,   1.350830e-17,   1.442085e-17,   1.525598e-17,
     &   1.386432e-

In [134]:
list(itertools.zip_longest([1 , 2, 3], [6, 7]))

[(1, 6), (2, 7), (3, None)]

In [135]:
itertools.zip_longest?